# Keyed dependency injection services

- The `IKeyedServiceProvider` interface.
- The `ServiceKeyAttribute` attribute, which can be used to inject the key that was used for registration/resolution in the constructor.
- The `FromKeyedServicesAttribute` attribute, which can be used on service constructor parameters to specify which keyed service to use.
- Various new extension methods for `IServiceCollection` to support keyed services, for example, `ServiceCollectionServiceExtensions.AddKeyedScoped`.
- The `ServiceProvider` implementation of `IKeyedServiceProvider`.

In [2]:
#r "nuget:Microsoft.DotNet.Interactive.AspNetCore, *-*"

Installed Packages Microsoft.DotNet.Interactive.AspNetCore, 1.0.0-beta.24101.2

Loading extension script from `C:\Users\DanielM\.nuget\packages\microsoft.dotnet.interactive.aspnetcore\1.0.0-beta.24101.2\interactive-extensions\dotnet\extension.dib`

In [3]:
#!aspnet
using Microsoft.Extensions.DependencyInjection;

var builder = WebApplication.CreateBuilder();
builder.Services.AddSingleton<BigCacheConsumer>();
builder.Services.AddSingleton<SmallCacheConsumer>();
builder.Services.AddKeyedSingleton<ICache, BigCache>("big");
builder.Services.AddKeyedSingleton<ICache, SmallCache>("small");
var app = builder.Build();
app.MapGet("/big", (BigCacheConsumer data) => data.GetData());
app.MapGet("/small", (SmallCacheConsumer data) => data.GetData());
app.MapGet("/big-cache", ([FromKeyedServices("big")] ICache cache) => cache.Get("data"));
app.MapGet("/small-cache", (HttpContext httpContext) => httpContext.RequestServices.GetRequiredKeyedService<ICache>("small").Get("data"));
app.Run();

class BigCacheConsumer([FromKeyedServices("big")] ICache cache)
{
    public object? GetData() => cache.Get("data");
}

class SmallCacheConsumer(IServiceProvider serviceProvider)
{
    public object? GetData() => serviceProvider.GetRequiredKeyedService<ICache>("small").Get("data");
}

public interface ICache
{
    object Get(string key);
}

public class BigCache : ICache
{
    public object Get(string key) => $"Resolving {key} from big cache.";
}

public class SmallCache : ICache
{
    public object Get(string key) => $"Resolving {key} from small cache.";
}

info: Microsoft.Hosting.Lifetime[14]
      Now listening on: http://localhost:5000
info: Microsoft.Hosting.Lifetime[0]
      Application started. Press Ctrl+C to shut down.
info: Microsoft.Hosting.Lifetime[0]
      Hosting environment: Local
info: Microsoft.Hosting.Lifetime[0]
      Content root path: c:\Users\DanielM\Documents\ISKA\Dotnet 8
info: Microsoft.AspNetCore.Hosting.Diagnostics[1]
      Request starting HTTP/1.1 GET http://localhost:5000/big - - -
info: Microsoft.AspNetCore.Routing.EndpointMiddleware[0]
      Executing endpoint 'HTTP: GET /big => <<Initialize>>b__0_0'
info: Microsoft.AspNetCore.Routing.EndpointMiddleware[1]
      Executed endpoint 'HTTP: GET /big => <<Initialize>>b__0_0'
info: Microsoft.AspNetCore.Hosting.Diagnostics[2]
      Request finished HTTP/1.1 GET http://localhost:5000/big - 200 - text/plain;+charset=utf-8 246.4761ms
info: Microsoft.AspNetCore.Hosting.Diagnostics[1]
      Request starting HTTP/1.1 GET http://localhost:5000/favicon.ico - - -
info: Micr

Error: Command cancelled.

## Real world example

In [1]:
public class User {}

public interface IUserService {
    Task<User> GetUser(string userId);
    Task<IEnumerable<User>> GetUsers();
}

public interface IUserServiceAsUser : IUserService { }
public interface IUserServiceAsApp : IUserService { }

public class AADUserService : IUserService, IUserServiceAsUser, IUserServiceAsApp { }

services.AddScoped<IUserServiceAsApp>(sp => new AADUserService(/* app specific login service*/));
services.AddScoped<IUserServiceAsUser>(sp => new AADUserService(/* user specific login service*/));

// with keyed DI
builder.Services.AddKeyedScoped<IUserService>("app", sp => new AADUserService(/* app specific login service*/)));
builder.Services.AddKeyedScoped<IUserService>("user", sp => new AADUserService(/* user specific login service*/));

public void GetUsers([FromKeyedServices("app")] IUserService userService) { } 
public void GetUsers([FromKeyedServices("user")] IUserService userService) { } 

Error: (18,79): error CS8635: Unexpected character sequence '...'
(18,81): error CS1003: Syntax error, ',' expected
(19,80): error CS8635: Unexpected character sequence '...'
(19,82): error CS1003: Syntax error, ',' expected